In [1]:
import os

from set_secrets import set_secrets
from src.src import get_query_dict, query_all_data

if "src_secret.py" in os.listdir("src"):
    from src.src_secret import secret_function

In [2]:
performance_id, mode_of_sale_id, constituent_id, source_id = (
    os.environ["PERFORMANCE_ID"],
    os.environ["MODE_OF_SALE_ID"],
    os.environ["CONSTITUENT_ID"],
    os.environ["SOURCE_ID"],
)

In [3]:
query_dict = get_query_dict(
    performance_id=performance_id,
    constituent_id=constituent_id,
    mode_of_sale_id=mode_of_sale_id,
    source_id=source_id,
)
all_data = query_all_data(query_dict)

Queried data keys: dict_keys(['seats', 'prices', 'zone_ids', 'price_types'])


In [ ]:
if "secret_function" in globals():
    secret_function(query_dict)

In [18]:
seats_price_df, prices_df, zones_df, price_types_df = (
    all_data["seats_price"],
    all_data["prices"],
    all_data["zones"],
    all_data["price_types"],
)

In [24]:
seats_price_df = seats_price_df.assign(seat_available=(~seats_price_df.SeatStatusId.isin([13, 3, 4, 6])))

In [25]:
seats_price_df.Price = seats_price_df.Price.where(seats_price_df.seat_available, None)

In [30]:
seats_price_df.query("SeatRow == 'WC' and SeatNumber == '27'")

,SeatId,SectionId,SeatRow,SeatNumber,ZoneId,AllocationId,SeatTypeId,LogicalSeatRow,LogicalSeatNumber,XPosition,...,PerformanceId,PackageId,PriceTypeId,Price,Enabled,ZoneGroupId,ZoneName,zone_hierarchy,seat_available,Size
1063,686064,278,WC,27,81860,274,1,2360,8,62,...,54659,0,7,NaN,True,5,Balcony,3,False,1
767,686948,271,WC,27,81824,273,1,2280,4,66,...,54659,0,7,NaN,True,3,Stalls Circle,1,False,1


In [28]:
import plotly.express as px

seats_price_df["Size"] = 1  # Dummy constant size for scatter plot

fig = px.scatter(
    seats_price_df,
    x="XPosition",
    y="YPosition",
    color="Price",
    size="Size",
    size_max=8.5,
    color_continuous_scale=[
        [0.0, "rgb(136,204,39)"],
        [0.5, "rgb(253,149,45)"],
        [0.77, "rgb(69,104,255)"],
        [0.89, "rgb(47,248,254)"],
        [0.96, "rgb(251,32,203)"],
        [1.0, "rgb(250,53,38)"],
    ],
    hover_data=["Price", "ZoneName", "SeatRow", "SeatNumber"],
    template="simple_white",
)
fig.update_xaxes(
    showticklabels=False, showline=False, zeroline=False, ticks="", title=""
)
fig.update_yaxes(
    showticklabels=False, showline=False, zeroline=False, ticks="", title=""
)
fig.update_layout(
    autosize=False,
    width=800,
    height=900,
    coloraxis=dict(
        colorbar=dict(
            tickmode="linear", dtick=25, ticks="inside", len=0.5, thickness=15
        )
    ),
    margin=dict(l=0, r=0, b=0, t=0, pad=0),
    # coloraxis_showscale=False,
)
fig.update_coloraxes(
    colorbar=dict(
        tickmode="linear", dtick=25, ticks="inside", orientation="h", x=0.46, y=0.94
    )
)
fig.add_shape(
    type="rect",
    x0=58.5,
    y0=-8,
    x1=22.5,
    y1=-3,
    line=dict(color="black", width=2),
    fillcolor="white",
    opacity=1,
)
# Add text for stage
fig.add_annotation(
    x=40.5,
    y=-5.5,
    text="Stage",
    showarrow=False,
    font=dict(size=20, color="black"),
)
fig.update_traces(marker=dict(line=dict(width=0.5, color="DarkSlateGrey")))
# Set the legend horizontally at the bottom
fig.update_layout()
fig.show()

In [31]:
# Save the figure as a static image
fig.write_image("ROH_hall.png", scale=3)